# Iterative Solvers and Sparse Matrices

# Iterative Solvers and Sparse Matrices
We are looking at the traditional sparse solvers in Chapter 3.  Largely because they are easy to implement and monitor the convergence of.

We want to solve $A x=b$ and we split a matrix $A=L+D+U$ into Lower Triangular, Diagonal, and Upper 
Triangular parts so we want to solve $(L+D+U) x^k = b$ where we have written $x^k$ to remind us that 
our target is an iterative solver we hope to have $x_k \rightarrow x^\infty = A^{-1} b$.

## Jacobi Iteration
The Jacobi iteration moves all but the diagonal portion to the other right side while "lagging" the iteration 
on the right hand side i.e. 
$
D x^{k+1) = b -  U x^k -L x^k 
$
The Jacobi iteration is 
$$
x^{k+1) = D^{-1} \left( b -  (U+L) x^k \right) 
$$

## Gauss-Seidell Iteration
The Gauss-Seidell iteration moves only the lower triangular portion to the right side while "lagging" the iteration 
$
(D+U) x^{k+1) = b -L x^k 
$
The GS iteration is 
$$
x^{k+1) = (D+U)^{-1} \left( b -  L x^k \right). 
$$

## Reasonably Efficient Implementation in Julia
These are both easy to implement in Julia.  We are going to look at some help to try to make sure we are as 
efficient as possible.  I am using our purely stripey matrix as an example. You should think what happensif
IO switch to the periodic version with the extreme off-diagonals. 

In [36]:
using SparseArrays, LinearAlgebra
m=124; dia=-4.0;
A = spzeros(m,m)
for i in 2:m
    A[i,i]=dia; A[i-1,i]=A[i,i-1]=1.0
end
A[1,1]=dia
A[m,1]=A[m,1]=1.0
L=tril(A,-1); U=triu(A,1); D=tril(triu(A)) 
DU=D+U
b=rand(m);x=rand(m)
@time BackSlashx=A\b
@time x=DU\(b-L*x)
1

  0.000120 seconds (67 allocations: 156.758 KiB)
  0.000010 seconds (3 allocations: 3.188 KiB)


1

## Doing Things with Sparse Matrices

All you can really do with a native sparse matrix is blindingly fast multiplication! There are however quite a lot of things you can do with just multiplication. 

In [74]:
x=rand(m)
for i in 1:123
    x=A*x
    x=x/norm(x)
end
 [x (eigen(Matrix(A)).vectors)[:,1]]

12×2 Matrix{Float64}:
  0.0894609   0.0938673
 -0.174471   -0.182279
  0.250668    0.260098
 -0.313895   -0.322801
  0.360387    0.366744
 -0.387003   -0.389372
  0.391514    0.389372
 -0.372887   -0.366744
  0.331521    0.322801
 -0.269381   -0.260098
  0.189985    0.182279
 -0.0982212  -0.0938673

## Jacobi Iteration
Here is our first Iterative solver.  It is called the Jacobi iteration. 

In [76]:
x=rand(m); b=rand(m)
Aoff=A+2.0*I
for i in 1:1234
    x = -0.5*(b-Aoff*x)
end
[A*x b]

12×2 Matrix{Float64}:
 0.879035    0.879035
 0.478901    0.478901
 0.00512292  0.00512292
 0.611677    0.611677
 0.900986    0.900986
 0.893156    0.893156
 0.274658    0.274658
 0.931816    0.931816
 0.992835    0.992835
 0.721366    0.721366
 0.621596    0.621596
 0.619942    0.619942